## Vacancies in graphene: an application of adiabatic quantum optimization

Notebook to reproduce the <a href="https://arxiv.org/abs/2010.05803">results on graphene defects</a>

In [133]:
from pymatgen.core.structure import Molecule, Structure, Lattice
from pymatgen.core.surface import SlabGenerator, generate_all_slabs
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.analysis.adsorption import AdsorbateSiteFinder
from pymatgen.ext.matproj import MPRester
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.analysis.graphs import StructureGraph, ConnectedSite
from pymatgen.analysis.local_env import VoronoiNN

from ase.visualize import view
import copy
import time
import itertools

import numpy as np

## Create the structure 

In [134]:
#with MPRester("p5vAQV3F1QuxFcxVT") as m:    
    #graphene_mp = m.get_structure_by_material_id("mp-48")

In [135]:
lattice = np.array([[ 1.233862, -2.137112,  0.      ],
                   [ 1.233862,  2.137112,  0.      ],
                   [ 0.      ,  0.      ,  8.685038]])

In [136]:
graphene = Structure(lattice, species=['C','C'], coords=[[2/3, 1/3, 0. ],[1/3, 2/3, 0.]])
graphene = SpacegroupAnalyzer(graphene).get_conventional_standard_structure()

In [137]:
n_supercell = 3
scaling_matrix = np.identity(3)*n_supercell
scaling_matrix[2][2] = 1
graphene_supercell = copy.deepcopy(graphene)
graphene_supercell.make_supercell(scaling_matrix)
graphene_supercell

Structure Summary
Lattice
    abc : 7.4031724240988614 7.4031724240988614 8.685038
 angles : 90.0 90.0 120.00000000000001
 volume : 412.2285660225962
      A : 3.7015862120494307 -6.411335387866038 0.0
      B : 3.7015862120494307 6.411335387866038 0.0
      C : 0.0 0.0 8.685038
PeriodicSite: C (1.2339, 0.7124, 0.0000) [0.1111, 0.2222, 0.0000]
PeriodicSite: C (2.4677, 2.8495, 0.0000) [0.1111, 0.5556, 0.0000]
PeriodicSite: C (3.7016, 4.9866, 0.0000) [0.1111, 0.8889, 0.0000]
PeriodicSite: C (2.4677, -1.4247, 0.0000) [0.4444, 0.2222, 0.0000]
PeriodicSite: C (3.7016, 0.7124, 0.0000) [0.4444, 0.5556, 0.0000]
PeriodicSite: C (4.9354, 2.8495, 0.0000) [0.4444, 0.8889, 0.0000]
PeriodicSite: C (3.7016, -3.5619, 0.0000) [0.7778, 0.2222, 0.0000]
PeriodicSite: C (4.9354, -1.4247, 0.0000) [0.7778, 0.5556, 0.0000]
PeriodicSite: C (6.1693, 0.7124, 0.0000) [0.7778, 0.8889, 0.0000]
PeriodicSite: C (1.2339, -0.7124, 0.0000) [0.2222, 0.1111, 0.0000]
PeriodicSite: C (2.4677, 1.4247, 0.0000) [0.2222, 0.4444

In [138]:
view(AseAtomsAdaptor().get_atoms(graphene_supercell))

<Popen: returncode: None args: ['/Users/brunocamino/miniconda3/envs/qc/bin/p...>

## Map the structure to a graph

In [139]:
graphene_graph = StructureGraph.with_local_env_strategy(graphene_supercell,strategy=VoronoiNN())

## Build the adjacency matrix

#### Periodic boundary conditions

In [163]:
distance_matrix_pbc = graphene_supercell.distance_matrix
shells = np.unique(distance_matrix_pbc[0])
np.sum(np.round(distance_matrix_pbc,5) == np.round(shells[1],5), axis=1)

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [164]:
adjacency_matrix = np.round(distance_matrix_pbc,5) == np.round(shells[1],5)
num_neighbours = np.sum(adjacency_matrix, axis=1)

In [165]:
adjacency_matrix

array([[False, False, False, False, False, False, False, False, False,
         True,  True, False, False, False, False,  True, False, False],
       [False, False, False, False, False, False, False, False, False,
        False,  True,  True, False, False, False, False,  True, False],
       [False, False, False, False, False, False, False, False, False,
         True, False,  True, False, False, False, False, False,  True],
       [False, False, False, False, False, False, False, False, False,
         True, False, False,  True,  True, False, False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False,  True, False, False,  True,  True, False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False,  True,  True, False,  True, False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False,  True, False, False,  True,  True, False],

## Define the binary variables that encode the problem

In [228]:
x = np.array([[1]*graphene_supercell.num_sites]*graphene_supercell.num_sites)
np.fill_diagonal(x,0)

In [229]:
x = x[0]
-np.matmul(x,adjacency_matrix)

array([-3, -3, -3, -3, -3, -3, -3, -3, -3, -2, -2, -3, -3, -3, -3, -2, -3,
       -3])

#### Classical

In [169]:
E_tmp = -np.matmul(x,adjacency_matrix)
np.sum(x*E_tmp,axis=1)

array([-48, -48, -48, -48, -48, -48, -48, -48, -48, -48, -48, -48, -48,
       -48, -48, -48, -48, -48])

### Classical full calculation

In [145]:
time_0 = time.time()

In [146]:
X_classical_1 = list(itertools.product([0, 1], repeat=18))
X_classical_2 = np.array(X_classical_1)

In [147]:
sorting = np.argsort(np.sum(X_classical_2,axis=1))

In [148]:
X_classical = X_classical_2[sorting]

In [149]:
E_tmp = -np.matmul(X_classical,adjacency_matrix)
E_classical = np.sum(X_classical*E_tmp,axis=1)

In [150]:
np.unique(E_classical)

array([-42, -40, -38, -36, -34, -32, -30, -28, -26, -24, -22, -20, -18,
       -16, -14, -12, -10,  -8,  -6,  -4,  -2,   0])

In [151]:
print(time.time()-time_0)

0.39359593391418457


# Quantum Adiabatic Computing

## Binary Quadratic Model

### Unconstrained

In [244]:
from dimod import BinaryQuadraticModel
from dwave.system import EmbeddingComposite, DWaveSampler

X = np.arange(graphene_supercell.num_sites)

Q = np.triu(-adjacency_matrix.astype(int),0)
bqm = BinaryQuadraticModel.from_qubo(Q)

In [245]:
X.tolist()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

In [247]:
sampler = EmbeddingComposite(DWaveSampler())
sampleset_unconstrained = sampler.sample(bqm, num_reads = 100, 
                                         label='Graphene 3x3 unconstrained')

In [169]:
df_unconstrained = sampleset_unconstrained.to_pandas_dataframe()

In [541]:
df_unconstrained.loc[:, df_unconstrained.columns != 'chain_break_fraction']

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,energy,num_occurrences
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-27.0,100


In [194]:
df = copy.deepcopy(df_unconstrained)
min_energy = min(df['energy' ])
n_atoms = graphene_supercell.num_sites 
df_unconstrained_low = df[df['energy' ] == min_energy].iloc[: , :n_atoms]
array_un = df[df['energy' ] == min_energy].iloc[: , :n_atoms].to_numpy()

In [195]:
array_un

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int8)

#### Lower Q

In [559]:
from dimod import BinaryQuadraticModel
from dwave.system import EmbeddingComposite, DWaveSampler

X = np.arange(graphene_supercell.num_sites)

Q = np.triu(-0.0001*adjacency_matrix.astype(int),0)
bqm = BinaryQuadraticModel.from_qubo(Q)

In [560]:
sampler = EmbeddingComposite(DWaveSampler())
sampleset_unconstrained = sampler.sample(bqm, num_reads = 100, 
                                         label='Graphene 3x3 unconstrained')

In [561]:
df_unconstrained = sampleset_unconstrained.to_pandas_dataframe()

In [562]:
df_unconstrained.loc[:, df_unconstrained.columns != 'chain_break_fraction']

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,energy,num_occurrences
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-0.0027,1
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-0.0027,1
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-0.0027,1
3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-0.0027,1
4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-0.0027,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-0.0027,1
96,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-0.0027,1
97,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-0.0027,1
98,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-0.0027,1


In [563]:
df = copy.deepcopy(df_unconstrained)
min_energy = min(df['energy' ])
n_atoms = graphene_supercell.num_sites 
df_unconstrained_low = df[df['energy' ] == min_energy].iloc[: , :n_atoms]
array_un = df[df['energy' ] == min_energy].iloc[: , :n_atoms].to_numpy()

In [564]:
np.unique(array_un,axis=0, return_counts=True)

(array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int8),
 array([100]))

### Constrained

#### 1 vacancy

In [264]:
from dimod import BinaryQuadraticModel
from dwave.system import EmbeddingComposite, DWaveSampler

X = np.arange(graphene_supercell.num_sites)

Q = np.triu(-adjacency_matrix.astype(int),0)
bqm = BinaryQuadraticModel.from_qubo(Q)

c_n_vacancies = [(i,1) for i in X]

num_vacancies = 1

bqm.add_linear_equality_constraint(
        c_n_vacancies,
        constant= -(graphene_supercell.num_sites-num_vacancies),
        lagrange_multiplier = 1000
        )

In [265]:
sampler = EmbeddingComposite(DWaveSampler())
sampleset_1v = sampler.sample(bqm, num_reads = 1000, label='Example - Simple Ocean Programs: BQM')

In [266]:
df_1v = sampleset_1v.to_pandas_dataframe()

In [267]:
df_1v

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,chain_break_fraction,energy,num_occurrences
0,1,1,1,1,1,1,1,1,1,1,...,1,1,0,1,1,1,1,0.000000,-24.0,6
1,1,1,1,1,0,1,1,1,1,1,...,1,1,1,1,1,1,1,0.000000,-24.0,26
2,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,0.000000,-24.0,9
3,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,1,0.000000,-24.0,7
4,1,1,1,1,1,1,0,1,1,1,...,1,1,1,1,1,1,1,0.000000,-24.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,1,1,0,1,1,1,1,1,1,0,...,1,1,1,1,1,1,1,0.055556,978.0,1
286,1,0,1,1,1,1,1,1,1,1,...,1,0,0,1,1,1,1,0.055556,3982.0,1
287,1,1,1,1,1,0,1,1,1,1,...,1,1,0,1,1,1,1,0.055556,3982.0,1
288,1,1,1,1,1,1,1,1,1,0,...,0,1,1,1,1,0,1,0.000000,8985.0,1


In [515]:
df_1v[df_1v['energy'] > -24.0]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,energy,num_occurrences
75,1,0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,978.0,1
360,1,1,1,1,1,1,1,1,0,1,1,1,1,0,1,1,1,1,979.0,1


In [504]:
df = copy.deepcopy(df_1v)
min_energy = min(df['energy' ])
n_atoms = graphene_supercell.num_sites 
df_1v_low = df[df['energy' ] == min_energy].iloc[: , :n_atoms]
array_1v = df[df['energy' ] == min_energy].iloc[: , :n_atoms].to_numpy()

#### 2 vacancies

In [223]:
from dimod import BinaryQuadraticModel
from dwave.system import EmbeddingComposite, DWaveSampler

X = np.arange(graphene_supercell.num_sites)

Q = np.triu(-adjacency_matrix.astype(int),0)
bqm = BinaryQuadraticModel.from_qubo(Q)

c_n_vacancies = [(i,1) for i in X]

num_vacancies = 2

bqm.add_linear_equality_constraint(
        c_n_vacancies,
        constant= -(graphene_supercell.num_sites-num_vacancies),
        lagrange_multiplier = 1000
        )

In [225]:
sampler = EmbeddingComposite(DWaveSampler())
#sampleset_2v = sampler.sample(bqm, num_reads = 1000, label='Example - Simple Ocean Programs: BQM')

In [226]:
df_2v = sampleset_2v.to_pandas_dataframe()

In [539]:
df_2v[df_2v['energy'] <0 ]

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,chain_break_fraction,energy,num_occurrences
0,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,1,1,0,0.000000,-22.0,1
1,1,1,1,1,1,1,1,0,1,1,...,1,1,0,1,1,1,1,0.000000,-22.0,4
2,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,1,0,1,0.000000,-22.0,4
3,1,1,1,1,1,1,1,1,0,1,...,1,1,1,1,1,1,0,0.000000,-22.0,7
4,1,1,0,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,0.000000,-22.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559,1,1,1,1,1,0,1,1,1,0,...,1,1,1,1,1,1,1,0.055556,-21.0,1
560,1,1,1,1,1,1,0,1,0,1,...,1,1,1,1,1,1,1,0.055556,-21.0,1
561,1,1,1,0,1,1,1,1,1,1,...,1,1,1,1,1,1,0,0.055556,-21.0,1
562,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,1,1,0.055556,-21.0,1


In [528]:
df = copy.deepcopy(df_2v)
min_energy = min(df['energy' ])
energies_unique = np.unique(df['energy' ])
n_atoms = graphene_supercell.num_sites 
df_2v_low = df[df['energy' ] == min_energy].iloc[: , :n_atoms]
array_2v = df[df['energy' ] == min_energy].iloc[: , :n_atoms].to_numpy()

In [229]:
df.head(50)

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,chain_break_fraction,energy,num_occurrences
0,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,1,1,0,0.0,-22.0,1
1,1,1,1,1,1,1,1,0,1,1,...,1,1,0,1,1,1,1,0.0,-22.0,4
2,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,1,0,1,0.0,-22.0,4
3,1,1,1,1,1,1,1,1,0,1,...,1,1,1,1,1,1,0,0.0,-22.0,7
4,1,1,0,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,0.0,-22.0,1
5,1,0,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,1,0.0,-22.0,4
6,1,0,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,0.0,-22.0,1
7,1,1,0,1,1,1,1,1,1,0,...,1,1,1,1,1,1,1,0.0,-22.0,3
8,1,1,1,1,1,0,1,1,1,1,...,0,1,1,1,1,1,1,0.0,-22.0,1
9,1,1,1,1,1,0,1,1,1,1,...,1,1,1,0,1,1,1,0.0,-22.0,2


In [529]:
array_2v = df[df['energy' ] <0.].iloc[: , :n_atoms].to_numpy()

#### 3 vacancies

In [442]:
from dimod import BinaryQuadraticModel
from dwave.system import EmbeddingComposite, DWaveSampler

X = np.arange(graphene_supercell.num_sites)

Q = np.triu(-adjacency_matrix.astype(int),0)
bqm = BinaryQuadraticModel.from_qubo(Q)

c_n_vacancies = [(i,1) for i in X]

num_vacancies = 3

bqm.add_linear_equality_constraint(
        c_n_vacancies,
        constant= -(graphene_supercell.num_sites-num_vacancies),
        lagrange_multiplier = 1000
        )

In [443]:
sampler = EmbeddingComposite(DWaveSampler())
#sampleset_3v = sampler.sample(bqm, num_reads = 1000, label='Example - Simple Ocean Programs: BQM')

In [444]:
df_3v = sampleset_3v.to_pandas_dataframe()

In [445]:
df_3v

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,energy,num_occurrences
0,1,0,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,-18.0,1
1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,-18.0,1
2,0,1,1,1,1,0,1,1,1,1,0,1,1,1,1,1,1,1,-19.0,1
3,1,0,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,-18.0,1
4,0,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,1,-18.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1,1,1,0,1,0,1,1,1,1,1,1,1,0,1,1,1,1,-19.0,1
996,1,1,1,1,0,1,1,1,1,1,1,1,1,0,1,0,1,1,-19.0,1
997,1,1,1,0,0,1,1,0,1,1,1,1,1,1,1,1,1,1,-18.0,1
998,0,1,1,1,1,0,1,1,1,1,1,1,1,1,1,0,1,1,-19.0,1


In [452]:
df = copy.deepcopy(df_3v)
min_energy = min(df['energy' ])
energies_unique = np.unique(df['energy' ])
n_atoms = graphene_supercell.num_sites 
df_3v_low = df[df['energy' ] == min_energy].iloc[: , :n_atoms]
#array_3v = df[df['energy' ] == min_energy].iloc[: , :n_atoms].to_numpy()

In [447]:
df.head(50)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,energy,num_occurrences
0,1,0,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,-18.0,1
1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,-18.0,1
2,0,1,1,1,1,0,1,1,1,1,0,1,1,1,1,1,1,1,-19.0,1
3,1,0,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,-18.0,1
4,0,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,1,-18.0,1
5,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,0,1,-18.0,1
6,1,1,1,0,1,1,1,0,1,1,1,1,1,1,1,1,1,0,-19.0,1
7,0,1,1,1,1,1,1,1,1,1,1,0,1,1,0,1,1,1,-18.0,1
8,0,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,-19.0,1
9,1,1,1,1,1,1,1,0,1,1,0,1,1,0,1,1,1,1,-19.0,1


In [461]:
array_3v = df[df['energy' ] <0.].iloc[: , :n_atoms].to_numpy()

In [457]:
array_3v = df.to_numpy()

In [460]:
np.unique(array_3v[:,18],return_counts=True,return_index=True)

(array([-20., -19., -18., 984.]),
 array([ 19,   2,   0, 599]),
 array([ 54, 392, 553,   1]))

### Add num of bonds constraint

In [284]:
from dimod import BinaryQuadraticModel
from dwave.system import EmbeddingComposite, DWaveSampler

X = np.arange(graphene_supercell.num_sites)

Q = np.triu(-adjacency_matrix.astype(int),0)
bqm = BinaryQuadraticModel.from_qubo(Q)

c_n_vacancies = [(i,1) for i in X]

num_vacancies = 2

bqm.add_linear_equality_constraint(
        c_n_vacancies,
        constant= -(graphene_supercell.num_sites-num_vacancies),
        lagrange_multiplier = 1000
        )

'''for i in range(graphene_supercell.num_sites):
    c_n_bonds = [(i,adjacency_matrix.astype(int)[j,i]) for j in X]
    bqm.add_linear_inequality_constraint(
        c_n_bonds,
        lb= - 2, 
        lagrange_multiplier=1,
        label = 'Min bonds_'+str(i)
    )'''

"for i in range(graphene_supercell.num_sites):\n    c_n_bonds = [(i,adjacency_matrix.astype(int)[j,i]) for j in X]\n    bqm.add_linear_inequality_constraint(\n        c_n_bonds,\n        lb= - 2, \n        lagrange_multiplier=1,\n        label = 'Min bonds_'+str(i)\n    )"

In [285]:
initial = bqm.to_numpy_matrix()

/var/folders/f2/8kc7y9697m59bwltxjd42y300000gn/T/ipykernel_89794/926658262.py:1: DeprecationWarning: bqm.to_numpy_matrix() is deprecated since dimod 0.10.0 and will be removed in 0.12.0
  initial = bqm.to_numpy_matrix()


In [286]:
final - initial

array([[9., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 9., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 9., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 9., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 9., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 9., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 9., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 9., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 9., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 9., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 9., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0

In [287]:
sampler = EmbeddingComposite(DWaveSampler())
sampleset_2v_b = sampler.sample(bqm, num_reads = 1000, label='2 vacancies, min bonds')

In [288]:
sampleset_2v_b_df = sampleset_3v_b.to_pandas_dataframe()

In [291]:
sampleset_2v_b_df[sampleset_2v_b_df['energy'] < 1000]

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,chain_break_fraction,energy,num_occurrences
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,1,1,0.000000,-3.0,1
1,1,1,1,1,1,1,1,1,1,0,...,1,1,1,1,0,1,1,0.000000,-3.0,2
2,1,1,1,1,0,1,1,1,1,1,...,1,1,1,1,1,1,1,0.000000,50.0,2
3,1,0,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,0.000000,50.0,1
4,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,1,1,0.000000,50.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,1,1,1,1,1,1,1,1,1,1,...,1,1,0,1,1,1,0,0.055556,141.0,1
641,1,1,1,0,1,1,1,1,1,1,...,1,1,1,1,1,0,1,0.055556,141.0,1
648,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,1,0,0.055556,51.0,1
649,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,0,1,0.055556,51.0,1


In [253]:
sampleset_3v_b_df[sampleset_3v_b_df['chain_break_fraction' == 0.]]

,1,1,0,0,1,1,0,0,0,0,...,1,0,1,1,1,0,0,1,1,1
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,0,0,1,1,0,0,1,1,1,1,...,0,1,0,0,0,1,1,0,0,0
2,1,1,0,0,1,1,0,0,0,0,...,1,0,1,1,1,0,0,1,1,1
3,1,1,0,0,1,1,0,0,0,0,...,1,0,1,1,1,0,0,1,1,1
4,0,0,1,1,0,0,1,1,1,1,...,0,1,0,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
774,1,1,0,0,1,1,0,0,0,0,...,1,0,1,1,1,0,0,1,1,1
775,1,1,0,0,1,1,0,0,0,0,...,1,0,1,1,1,0,0,1,1,1
776,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
777,0,0,1,1,0,0,1,1,1,1,...,0,1,0,0,0,1,1,0,0,0


## Constrained Quadratic Model

In [212]:
from dimod import ConstrainedQuadraticModel, Binary, quicksum
from dwave.system import LeapHybridCQMSampler

cqm = ConstrainedQuadraticModel()

atoms = [Binary(i) for i in range(graphene_supercell.num_sites)]

atoms = np.array(atoms)

Q = np.triu(-adjacency_matrix.astype(int),0)
bqm = BinaryQuadraticModel.from_qubo(Q)

#set the objective (minimise the energy of the adjacency matrix)

cqm.set_objective(bqm)

num_vacancies = 3

#set the constraint: number of vacancies
cqm.add_constraint(quicksum(atoms[i] for i in range(graphene_supercell.num_sites)) \
                   == (graphene_supercell.num_sites - num_vacancies), label='Num vacancies')

'Num vacancies'

### Constaint min n bonds

In [215]:
for i in range(graphene_supercell.num_sites):
    cqm.add_constraint(quicksum(atoms[0]*adjacency_matrix[0]) \
                   >= 2, label='Min_coord_'+str(i))

In [213]:
cqm_sampler = LeapHybridCQMSampler()
sampleset = cqm_sampler.sample_cqm(cqm,time_limit=5)

In [220]:
sampleset.to_pandas_dataframe()

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,energy,is_feasible,is_satisfied,num_occurrences
0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,-22.0,False,False,1
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,-22.0,False,False,1
2,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,-22.0,False,False,1
3,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,1.0,1.0,1.0,1.0,-22.0,False,False,1
4,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,-22.0,False,False,1
5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,-22.0,False,False,1
6,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,1.0,1.0,1.0,1.0,-22.0,False,False,1
7,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,-22.0,False,False,1
8,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,-22.0,False,False,1
9,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,-22.0,False,False,1


In [216]:
cqm_sampler = LeapHybridCQMSampler()
sampleset_bonds = cqm_sampler.sample_cqm(cqm,time_limit=5)

In [224]:
sampleset_bonds.aggregate()

SampleSet(rec.array([([1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.], -20., 6,  True, [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True]),
           ([1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1.], -20., 1,  True, [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True]),
           ([1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.], -20., 2,  True, [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True]),
           ([0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1.], -20., 2, False, [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False,  True, False, False]),
           ([1., 1., 0., 1., 1., 1., 1

### Analyse structures

In [248]:
vacancy_pos = np.where(array_2v == 0)
vacancy_pos

(array([  0,   0,   1,   1,   2,   2,   3,   3,   4,   4,   5,   5,   6,
          6,   7,   7,   8,   8,   9,   9,  10,  10,  11,  11,  12,  12,
         13,  13,  14,  14,  15,  15,  16,  16,  17,  17,  18,  18,  19,
         19,  20,  20,  21,  21,  22,  22,  23,  23,  24,  24,  25,  25,
         26,  26,  27,  27,  28,  28,  29,  29,  30,  30,  31,  31,  32,
         32,  33,  33,  34,  34,  35,  35,  36,  36,  37,  37,  38,  38,
         39,  39,  40,  40,  41,  41,  42,  42,  43,  43,  44,  44,  45,
         45,  46,  46,  47,  47,  48,  48,  49,  49,  50,  50,  51,  51,
         52,  52,  53,  53,  54,  54,  55,  55,  56,  56,  57,  57,  58,
         58,  59,  59,  60,  60,  61,  61,  62,  62,  63,  63,  64,  64,
         65,  65,  66,  66,  67,  67,  68,  68,  69,  69,  70,  70,  71,
         71,  72,  72,  73,  73,  74,  74,  75,  75,  76,  76,  77,  77,
         78,  78,  79,  79,  80,  80,  81,  81,  82,  82,  83,  83,  84,
         84,  85,  85,  86,  86,  87,  87,  88,  88

In [578]:
structures = []
for i,pos in enumerate(array_3v):
    structure = copy.deepcopy(graphene_supercell)
    #print(structure.sites[np.where(pos == 0)[0][0]].distance(structure.sites[np.where(pos == 0)[0][1]]))
    #structure.remove_sites(np.where(pos == 0)[0])
    structure.replace(np.where(pos == 0)[0][0],1)
    structure.replace(np.where(pos == 0)[0][1],1)
    structure.replace(np.where(pos == 0)[0][2],1)
    structures.append(structure)

In [579]:
#a = SpacegroupAnalyzer(structures[0]).get_conventional_standard_structure()
view(AseAtomsAdaptor().get_atoms(structures[-3]))

<Popen: returncode: None args: ['/Users/brunocamino/miniconda3/envs/qc/bin/p...>

In [465]:
array_3v[0]

array([1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1], dtype=int8)

# Structural analysis

In [280]:
def cart2sph(x, y, z):
    hxy = np.hypot(x, y)
    r = np.hypot(hxy, z)
    el = np.arctan2(z, hxy)
    az = np.arctan2(y, x)
    if np.around(az,6) ==  np.around(2*np.pi,6) \
    or np.around(az,6) ==  -np.around(2*np.pi,6):
        az = 0.
    if np.around(az,6) < 0.:
        az = np.round(2*np.pi+az,6)
    return [round(az,6), round(el,6), round(r,6)]

In [281]:
max_shell = 2
centered_sph_coords = []
centered_sph_coords_structure = []
neighbours_spatial_dist = []
neighbours_spatial_dist_all = []
import time
#time0 = time.time()
#for k,structure in enumerate(structures[0:100]): #TMP only calculate 100 structures
for k,structure in enumerate(structures): 
    time0 = time.time()
    neighbours_spatial_dist = []
    
    for j in range(structure.num_sites):
        centered_sph_coords = []
        neighbours_spatial_dist_atom = []
        
        for n in range(max_shell+1):
            atom_indices = np.where(np.round(structure.distance_matrix[j],5) == np.round(shells[n],5))[0].tolist()
            centered_sph_coords = []
            for i in atom_indices:

                translation_vector = structure.sites[j].distance_and_image(structure.sites[i])[1]
                new_cart_coords = structure.cart_coords[i]+(translation_vector*structure.lattice.abc)
                centered_cart_coords = new_cart_coords-structure.cart_coords[j] 

                centered_sph_coords.append(cart2sph(centered_cart_coords[0],centered_cart_coords[1],centered_cart_coords[2]))        

            spatial_distribution = np.argsort(np.array(centered_sph_coords)[:,1]*10 +\
                                              np.array(centered_sph_coords)[:,0])


            neighbours_spatial_dist_atom.extend((np.array(structure.atomic_numbers)[np.array(atom_indices)[spatial_distribution]]).tolist())
        neighbours_spatial_dist.append(neighbours_spatial_dist_atom)
    #print(time.time()-time0)
    neighbours_spatial_dist_all.append(neighbours_spatial_dist) 
#neighbours_spatial_dist_all = np.array(neighbours_spatial_dist_all) 

In [282]:
neighbours_spatial_dist_all

[[[6, 6, 6, 6, 6, 6, 6],
  [6, 6, 6, 6, 6, 6, 6],
  [6, 6, 1, 6, 6, 1, 6],
  [6, 6, 6, 6, 6, 6, 6],
  [6, 6, 6, 6, 6, 6, 6],
  [6, 6, 6, 6, 6, 6, 6],
  [6, 6, 6, 6, 6, 6, 6],
  [1, 1, 6, 6, 1, 6, 6],
  [6, 6, 6, 1, 6, 6, 1],
  [6, 6, 6, 6, 6, 6, 6],
  [6, 6, 6, 6, 6, 6, 6],
  [6, 6, 6, 6, 6, 6, 6],
  [6, 6, 6, 6, 6, 6, 6],
  [6, 6, 6, 1, 6, 6, 1],
  [6, 6, 6, 6, 6, 6, 6],
  [6, 6, 6, 6, 6, 6, 6],
  [6, 6, 1, 6, 6, 1, 6],
  [1, 6, 6, 1, 6, 6, 1]],
 [[6, 6, 6, 6, 6, 6, 6],
  [6, 6, 6, 6, 6, 6, 6],
  [6, 6, 6, 6, 6, 6, 6],
  [6, 1, 6, 6, 1, 6, 6],
  [6, 6, 6, 1, 6, 6, 1],
  [6, 6, 6, 6, 6, 6, 6],
  [6, 6, 6, 6, 6, 6, 6],
  [1, 6, 1, 6, 6, 1, 6],
  [6, 6, 6, 6, 6, 6, 6],
  [6, 6, 6, 6, 6, 6, 6],
  [6, 6, 6, 6, 6, 6, 6],
  [6, 6, 6, 6, 6, 6, 6],
  [6, 6, 6, 6, 6, 6, 6],
  [1, 6, 6, 1, 6, 6, 1],
  [6, 6, 6, 6, 6, 6, 6],
  [6, 6, 6, 6, 6, 6, 6],
  [6, 6, 1, 6, 6, 1, 6],
  [6, 6, 6, 1, 6, 6, 1]],
 [[6, 6, 6, 6, 6, 6, 6],
  [6, 6, 1, 6, 6, 1, 6],
  [6, 6, 6, 6, 6, 6, 6],
  [6, 6, 6, 6, 6, 6, 6]

In [283]:
neighbours_spatial_dist_all_sorted = []
sorting = []

for k,structure in enumerate(structures):
    sorted_atoms = []
    for i in range(len(neighbours_spatial_dist_all[0])):
        sorted_atoms.append(int(''.join([str(x) for x in neighbours_spatial_dist_all[k][i]])))
    sorting.append(np.argsort(np.array(sorted_atoms)))    
    neighbours_spatial_dist_all_sorted.append((np.array(neighbours_spatial_dist_all)[k][np.argsort(np.array(sorted_atoms))]).tolist())
neighbours_spatial_dist_all_sorted = np.array(neighbours_spatial_dist_all_sorted)    


In [285]:
neighbours_spatial_dist_all_sorted[0]

array([[1, 1, 6, 6, 1, 6, 6],
       [1, 6, 6, 1, 6, 6, 1],
       [6, 6, 1, 6, 6, 1, 6],
       [6, 6, 1, 6, 6, 1, 6],
       [6, 6, 6, 1, 6, 6, 1],
       [6, 6, 6, 1, 6, 6, 1],
       [6, 6, 6, 6, 6, 6, 6],
       [6, 6, 6, 6, 6, 6, 6],
       [6, 6, 6, 6, 6, 6, 6],
       [6, 6, 6, 6, 6, 6, 6],
       [6, 6, 6, 6, 6, 6, 6],
       [6, 6, 6, 6, 6, 6, 6],
       [6, 6, 6, 6, 6, 6, 6],
       [6, 6, 6, 6, 6, 6, 6],
       [6, 6, 6, 6, 6, 6, 6],
       [6, 6, 6, 6, 6, 6, 6],
       [6, 6, 6, 6, 6, 6, 6],
       [6, 6, 6, 6, 6, 6, 6]])

In [286]:
neighbours_spatial_dist_all_sorted_sliced = neighbours_spatial_dist_all_sorted[:,:,1:]

In [287]:
neighbours_spatial_dist_all_sorted_sliced[0]

array([[1, 6, 6, 1, 6, 6],
       [6, 6, 1, 6, 6, 1],
       [6, 1, 6, 6, 1, 6],
       [6, 1, 6, 6, 1, 6],
       [6, 6, 1, 6, 6, 1],
       [6, 6, 1, 6, 6, 1],
       [6, 6, 6, 6, 6, 6],
       [6, 6, 6, 6, 6, 6],
       [6, 6, 6, 6, 6, 6],
       [6, 6, 6, 6, 6, 6],
       [6, 6, 6, 6, 6, 6],
       [6, 6, 6, 6, 6, 6],
       [6, 6, 6, 6, 6, 6],
       [6, 6, 6, 6, 6, 6],
       [6, 6, 6, 6, 6, 6],
       [6, 6, 6, 6, 6, 6],
       [6, 6, 6, 6, 6, 6],
       [6, 6, 6, 6, 6, 6]])

In [293]:
n_structures = neighbours_spatial_dist_all_sorted_sliced.shape[0]
vector_len = neighbours_spatial_dist_all_sorted_sliced.shape[1] * neighbours_spatial_dist_all_sorted_sliced.shape[2]
neighbours_spatial_dist_all_sorted_sliced_flat = \
np.reshape(neighbours_spatial_dist_all_sorted_sliced, [n_structures,vector_len])

In [301]:
neighbours_spatial_dist_all_sorted_sliced_flat[0]

array([1, 6, 6, 1, 6, 6, 6, 6, 1, 6, 6, 1, 6, 1, 6, 6, 1, 6, 6, 1, 6, 6,
       1, 6, 6, 6, 1, 6, 6, 1, 6, 6, 1, 6, 6, 1, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6])

In [314]:
a,b,c = np.unique(neighbours_spatial_dist_all_sorted_sliced_flat,axis=0,return_index=True,return_counts=True,)

In [315]:
for i in b:
    view(AseAtomsAdaptor().get_atoms(structures[i]))

# Simulated annealing
Only lowest energy solution

In [377]:
import neal

sampler = neal.SimulatedAnnealingSampler()

X = np.arange(graphene_supercell.num_sites)

Q = np.triu(-adjacency_matrix.astype(int),0)
bqm = BinaryQuadraticModel.from_qubo(Q)

c_n_vacancies = [(i,1) for i in X]

num_vacancies = 4

bqm.add_linear_equality_constraint(
        c_n_vacancies,
        constant= -(graphene_supercell.num_sites-num_vacancies),
        lagrange_multiplier = 1000
        )
sampleset = sampler.sample(bqm)

In [378]:
sampleset.to_pandas_dataframe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,energy,num_occurrences
0,1,1,1,0,0,1,1,1,1,0,1,1,1,1,1,1,0,1,-16.0,1
